In [1]:
import pandas as pd
import csv

import scrapy
import logging

from scrapy.item import Item, Field
from scrapy.crawler import CrawlerRunner
from scrapy.exporters import CsvItemExporter
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

from crochet import setup, wait_for

In [2]:
class EditalItem(Item):
    date = Field()
    tags = Field()
    title = Field()
    cover = Field()
    summary = Field()
    results = Field()
    pdf = Field()


class QuoteAllDialect(csv.excel):
    quoting = csv.QUOTE_ALL


class QuoteAllCsvItemExporter(CsvItemExporter):
    def __init__(self, *args, **kwargs):
        kwargs.update({'dialect': QuoteAllDialect})
        super(QuoteAllCsvItemExporter, self).__init__(*args, **kwargs)

In [3]:
class EditaisSpider(scrapy.Spider):
    name = 'secult'
    #max_pages = 2
    #start_urls = ['http://editais.cultura.ce.gov.br/category/editais/page/%s' % page for page in range(1, max_pages + 1)]
    start_urls = ['http://editais.cultura.ce.gov.br/category/editais/']
    delimiter = ';'
    quotechar = "'"
    custom_settings = {
        'FEEDS': {
            'editais.csv': {
                'format': 'csv',
                'overwrite': True
            }
        },
        'FEED_EXPORT_ENCODING': 'utf-8',
        'FEED_EXPORT_FIELDS': ['date', 'tags', 'title', 'cover', 'summary', 'pdf', 'results'],
        'FEED_EXPORTERS': {
            'csv': '__main__.QuoteAllCsvItemExporter',
        }

    }


    def parse(self, response):
        for edital in response.css('article'):
            url = edital.xpath('h2/a/@href').get()

            # Get date from url
            date = url.replace('http://editais.cultura.ce.gov.br/', '').split('/')
            # Format date yyyy-mm-dd
            date = f'{date[0]}-{date[1]}-{date[2]}'
            
            item = EditalItem()
            item['date'] = date
            item['tags'] = edital.xpath('p/a[@rel="category tag"]/text()').extract()

            yield scrapy.Request(url, callback=self.parse_edital, meta={'item':item})

        # Go to next page
        next_page = response.css('div.pagination').xpath('div/a/@href').get()
        if next_page is not None :
            yield response.follow(next_page, self.parse)


    def parse_edital(self, response):
        item = response.meta['item']
        item['title'] = response.css('h1.entry-title::text').get()
        item['cover'] = response.css('article img::attr("src")').get()
        item['summary'] = ''.join(response.css('div.et_pb_text_inner ::text').extract()).strip()
        item['pdf'] = response.css('a.et_pb_button::attr("href")').get()
        item['results'] = response.css('div.et_pb_row.et_pb_row_1 > div.et_pb_column.et_pb_column_1_3.et_pb_column_2.et_pb_css_mix_blend_mode_passthrough > div > div > div.et_pb_blurb_container > div a::attr("href")')[-1].extract()
        yield item
        
            

In [4]:
# Reactor restart
setup()
@wait_for(100)
def run_scraper():
    crawler = CrawlerRunner()
    d = crawler.crawl(EditaisSpider)
    return d

In [5]:
# Iniciando processo
run_scraper()

# process = CrawlerProcess()
# process.crawl(EditaisSpider)
# process.start()

In [6]:
df = pd.read_csv('editais.csv')
df

,date,tags,title,cover,summary,pdf,results
0,2021-02-03,"Mecenas,Sem categoria",XIII Edital Mecenas do Ceará,http://editais.cultura.ce.gov.br/wp-content/up...,O presente Edital tem como objeto a seleção de...,https://mapacultural.secult.ce.gov.br/files/op...,https://mapacultural.secult.ce.gov.br/files/op...
1,2020-03-05,"Cultura Infância,Editais,Sem categoria",Edital Cultura Infância 2020,http://editais.cultura.ce.gov.br/wp-content/up...,A Secretaria da Cultura do Estado do Ceará tor...,https://mapacultural.secult.ce.gov.br/files/op...,https://mapacultural.secult.ce.gov.br/files/op...
2,2021-11-19,"2022,Carnaval,Ceará da Paixão",CHAMADA PÚBLICA PARA SELEÇÃO DO IV SEMINÁRIO D...,http://editais.cultura.ce.gov.br/wp-content/up...,A Secretaria da Cultura do Ceará (Secult Ceará...,NaN,https://www.secult.ce.gov.br/2021/12/06/comuni...
3,2021-11-29,"2022,Carnaval,Sem categoria",XV EDITAL CEARÁ CICLO CARNAVALESCO – 2022,http://editais.cultura.ce.gov.br/wp-content/up...,A Secretaria da Cultura do Estado do Ceará (Se...,https://mapacultural.secult.ce.gov.br/files/op...,https://mapacultural.secult.ce.gov.br/files/op...
4,2019-01-03,"2019,Carnaval,Editais",XIII Edital Carnaval do Ceará,http://editais.cultura.ce.gov.br/wp-content/up...,"Na sua décima terceira edição, o Edital Carna...",http://mapa.cultura.ce.gov.br/files/opportunit...,http://mapa.cultura.ce.gov.br/files/opportunit...
5,2019-01-08,"2019,Carnaval",Chamada Pública para Credenciamento de Parecer...,http://editais.cultura.ce.gov.br/wp-content/up...,"O Secretaria da Cultura do estado do Ceará, co...",http://mapa.cultura.ce.gov.br/files/opportunit...,https://mapacultural.secult.ce.gov.br/files/op...
6,2019-07-15,"2019,Natal de Luz,Sem categoria",XVI Edital Ceará Ciclo Natalino 2019,http://editais.cultura.ce.gov.br/wp-content/up...,O XVI Edital Ceará Ciclo Natalino – 2019 é uma...,https://mapacultural.secult.ce.gov.br/files/op...,https://mapacultural.secult.ce.gov.br/files/op...
7,2019-08-27,"2019,Eleições CEPC",Eleições CEPC – 2020/2022,http://editais.cultura.ce.gov.br/wp-content/up...,O presente edital tem por objetivo a escolha d...,https://mapacultural.secult.ce.gov.br/files/op...,//mapacultural.secult.ce.gov.br/files/opportun...
8,2019-09-11,"2019,Incentivo às Artes,Sem categoria",XII Edital Ceará de Incentivo às Artes,http://editais.cultura.ce.gov.br/wp-content/up...,A Secretaria da Cultura do Estado do Ceará (Se...,https://mapacultural.secult.ce.gov.br/files/pr...,https://mapacultural.secult.ce.gov.br/files/op...
9,2019-10-01,"2019,Natal de Luz,Sem categoria",Edital XIV Mostra Estadual Ceará Ciclo Natalin...,http://editais.cultura.ce.gov.br/wp-content/up...,O Edital XIV Mostra Estadual Ceará Ciclo Natal...,https://mapacultural.secult.ce.gov.br/files/op...,https://mapacultural.secult.ce.gov.br/files/op...


In [7]:
edital_sample = df.sample()
edital_sample

,date,tags,title,cover,summary,pdf,results
1,2020-03-05,"Cultura Infância,Editais,Sem categoria",Edital Cultura Infância 2020,http://editais.cultura.ce.gov.br/wp-content/up...,A Secretaria da Cultura do Estado do Ceará tor...,https://mapacultural.secult.ce.gov.br/files/op...,https://mapacultural.secult.ce.gov.br/files/op...
